In [ ]:
!pip install langchain

!pip install cohere tiktoken openai


In [2]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://127.0.0.1:11434", api_key="not-needed")

completion = client.chat.completions.create(
  model="local-model", # this field is currently unused
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

NotFoundError: 404 page not found

In [3]:
import openai
import os

import pandas as pd

pd.set_option('display.max_colwidth', None)

file_name = "conference_session_info.csv"

df = pd.read_csv(file_name)



In [4]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path = file_name, encoding="utf8")

docs = loader.load()

In [5]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter

chunk_size = 128
chunk_overlap = 32

c_text_splitter = CharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = len,
    separator = ' '
)

r_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = chunk_size,
    chunk_overlap = chunk_overlap,
    length_function = len,
    add_start_index = True
)

In [6]:
pages = r_text_splitter.split_documents(docs)

In [7]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

from openai import OpenAI
client = OpenAI(base_url="http://127.0.0.1:11434/v1", api_key="not-needed")


os.environ["OPENAI_API_KEY"] = "not-needed"
os.environ["OPENAI_API_BASE"] = "http://127.0.0.1:11434/v1"


embedding = OpenAIEmbeddings()

persist_directory = 'persist_chroma'


/home/zanmatt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
vectordb = Chroma.from_documents(
    documents=pages,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

NotFoundError: 404 page not found

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

import langchain
langchain.verbose = True

llm_name = "gpt-3.5-turbo"

llm = ChatOpenAI(model_name=llm_name, temperature=1)